## Imports 

In [3]:
from hashlib import sha1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)

from stop_words import get_stop_words 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, make_pipeline                

## Load Cyberbullying Tweets csv file

In [4]:
tweets_df = pd.read_csv("controller/machine_learning/data/cyberbullying_tweets.csv")
print(tweets_df.head())
stop_words_set = set(get_stop_words('en'))



                                          tweet_text cyberbullying_type
0  In other words #katandandre, your food was cra...  not_cyberbullying
1  Why is #aussietv so white? #MKR #theblock #ImA...  not_cyberbullying
2  @XochitlSuckkks a classy whore? Or more red ve...  not_cyberbullying
3  @Jason_Gio meh. :P  thanks for the heads up, b...  not_cyberbullying
4  @RudhoeEnglish This is an ISIS account pretend...  not_cyberbullying


In [5]:
tweets_df["cyberbullying_type"].value_counts()
 

cyberbullying_type
religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: count, dtype: int64

## EDA

### Step 1: Modify Label if the value is 'not_cyberbullying' then 0, otherwise 1

In [6]:
tweets_df["cyberbullying_type"] = tweets_df["cyberbullying_type"].apply(lambda x: 0 if x == "not_cyberbullying" else 1)
tweets_df["cyberbullying_type"].value_counts()

cyberbullying_type
1    39747
0     7945
Name: count, dtype: int64

### Step 2: Each `tweet_text` Filtering (Also, Read **Further Notice**)
<hr>

`tweet_text` contains emoji or any other languages. To make model easily, we are going to stick to have only English.

[**Further Notice**] As we know, if we remove the any emoji or any other languages, we might lose useful information. Then, we might need to change this process in the future.

We also want to make all the text lowercase, remove stop words, and remove all links!

In [12]:
print(tweets_df.head())
tweets_df["tweet_text"] = tweets_df["tweet_text"].apply(lambda x: re.sub(r'http\S+', '', x))
tweets_df["tweet_text"] = tweets_df["tweet_text"].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))
tweets_df["tweet_text"] = tweets_df["tweet_text"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]+', '', x))
tweets_df["tweet_text"] = tweets_df["tweet_text"].apply(lambda x: x.lower())
tweets_df["tweet_text"] = tweets_df["tweet_text"].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words_set]))
print(tweets_df.head())

#Check this below
tweets_df["tweet_text"][21]


                                          tweet_text  cyberbullying_type
0            words katandandre food crapilicious mkr                   0
1  aussietv white mkr theblock imacelebrityau tod...                   0
2    xochitlsuckkks classy whore red velvet cupcakes                   0
3  jasongio meh p thanks heads concerned another ...                   0
4  rudhoeenglish isis account pretending kurdish ...                   0
                                          tweet_text  cyberbullying_type
0            words katandandre food crapilicious mkr                   0
1  aussietv white mkr theblock imacelebrityau tod...                   0
2    xochitlsuckkks classy whore red velvet cupcakes                   0
3  jasongio meh p thanks heads concerned another ...                   0
4  rudhoeenglish isis account pretending kurdish ...                   0


'kids love mohamad bin zayed city'

In [8]:
X = tweets_df["tweet_text"]
y = tweets_df["cyberbullying_type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)


## Preprocessing

### Logistic Regression

In [7]:
# 2. Create a vectorizer object with stop_words = "english"
countvec = CountVectorizer(stop_words="english")

# 3. Create a logisticRegression object
lr = LogisticRegression(max_iter=1000, random_state=123)
# lr = LogisticRegression()

# 4. Make a pipeline object
pipe = make_pipeline(countvec, lr)

pipe.fit(X_train, y_train)

# 5. Store the mean values of cross-validation scores
cv_score = cross_val_score(pipe, X_train, y_train).mean()
cv_score

0.861478885273956

### SVM RBF

In [8]:
# 2. Create a vectorizer object with stop_words = "english"
countvec = CountVectorizer(stop_words="english")

# 3. Create a SVM RBF object
svm = SVC(kernel="rbf", C=10, gamma=0.1)

# 4. Make a pipeline object
pipe = make_pipeline(countvec, svm)

# 5. Store the mean values of cross-validation scores
cv_score = cross_val_score(pipe, X_train, y_train).mean()
cv_score

0.8593820027400823

### RandomForest

In [11]:
# 2. Create a vectorizer object with stop_words = "english"
countvec = CountVectorizer(stop_words="english")

# 3. Create a RandomForest object
rf = RandomForestClassifier(n_jobs=-1, random_state=123)

# 4. Make a pipeline object
pipe = make_pipeline(countvec, rf)

# 5. Store the mean values of cross-validation scores
cv_score = cross_val_score(pipe, X_train, y_train).mean()
cv_score

test = "i love you"

pipe.fit(X_train, y_train)
print(tweets_df["tweet_text"][0])
result = pipe.predict_proba([tweets_df["tweet_text"][0]])
print(result)




words katandandre food crapilicious mkr
0.8560645769996855
[[0.95916667 0.04083333]]
